In [1]:
import os
import cv2
import numpy as np
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random


from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, Flatten, Dense, Dropout, Activation, Lambda, Cropping2D
from keras.callbacks import ModelCheckpoint, CSVLogger

Using TensorFlow backend.


In [2]:
CONFIG = {
    'batchsize': 32,
    'input_width': 320,
    'input_height': 160,
    'input_channels': 3,
    'correction': 0.2,
    'cropping': ((50,20), (0,0))
}

In [3]:
def load_and_split_data(data_path, test_size=0.2):
    with open(data_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        samples = [line for line in reader][1:]
    train_samples, validation_samples = train_test_split(samples, test_size=test_size, random_state=0)
    return train_samples, validation_samples

def generator(samples, img_path, multi_camera=False, augment_data=False, batch_size=32):
    num_samples = len(samples)
    while 1:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                center, left, right, steering, throttle, brake, speed = batch_sample
                
                steering_center = float(steering)
                center_image = cv2.imread(os.path.join(img_path, center))

                images.append(center_image)
                angles.append(steering_center)

                if multi_camera:
                    correction = CONFIG['correction']
                    steering_left = steering_center + correction
                    steering_right = steering_center - correction

                    left_image = cv2.imread(os.path.join(img_path, left))
                    right_image = cv2.imread(os.path.join(img_path, right))
                    
                    images.extend([center_image, left_image, right_image])
                    angles.extend([steering_center, steering_left, steering_right])
                
                if augment_data:
                    for image, measurement in zip(images, angles):
                        images.append(cv2.flip(image, 1))
                        angles.append(measurement*-1.0)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)

def nvidia_model(summary=True):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(CONFIG['input_height'], CONFIG['input_width'], CONFIG['input_channels'])))
    model.add(Cropping2D(cropping=CONFIG['cropping']))
    model.add(Convolution2D(24,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2),activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    if summary:
        model.summary()
    
    return model

In [4]:

train_samples, validation_samples = load_and_split_data(data_path='./data/data/driving_log.csv', test_size=0.2)

train_generator = generator(train_samples, img_path='./data/data', multi_camera=False, augment_data=False,  batch_size=CONFIG['batchsize'])
validation_generator = generator(validation_samples, img_path='./data/data', batch_size=CONFIG['batchsize'])

model = nvidia_model(summary=True)
model.compile(optimizer='adam', loss='mse')

# json dump of model architecture
with open('logs/model.json', 'w') as f:
    f.write(model.to_json())

# define callbacks to save history and weights
checkpointer = ModelCheckpoint('checkpoints/weights.{epoch:02d}-{val_loss:.3f}.hdf5')
logger = CSVLogger(filename='logs/history.csv')

history_object = model.fit_generator(train_generator, samples_per_epoch=CONFIG['batchsize']*300, 
                                    validation_data=validation_generator, nb_val_samples=CONFIG['batchsize']*100, 
                                    nb_epoch=3, verbose=1)

model.save('model.h')

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:49: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 33, 64)         36928     
__________